In [2]:
# MarxBot: Ask Marx Anything

# Project Overview
# A simple RAG (Retrieval-Augmented Generation) chatbot using TinyLlama,
# backed by a vector database created from classic Marxist texts (public domain).

# Requirements
# pip install langchain chromadb sentence-transformers langchain-community

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import (
    TextLoader,
    PyPDFLoader,
    UnstructuredPDFLoader,
    UnstructuredWordDocumentLoader
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
import os

In [4]:
# Step 1: Load Documents (.txt, .pdf, .docx)
doc_dir = "marxtexts"
texts = []
for filename in os.listdir(doc_dir):
    filepath = os.path.join(doc_dir, filename)

    if filename.endswith(".txt"):
        loader = TextLoader(filepath, encoding='utf-8')
        texts.extend(loader.load())

    elif filename.endswith(".pdf"):
        try:
            loader = PyPDFLoader(filepath)
            texts.extend(loader.load())
        except Exception:
            loader = UnstructuredPDFLoader(filepath)
            texts.extend(loader.load())

    elif filename.endswith(".docx"):
        loader = UnstructuredWordDocumentLoader(filepath)
        texts.extend(loader.load())

In [ ]:
# Step 2: Split & Embed
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(texts)

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(chunks, embedding=embedding_model, persist_directory="marx_index")
vectorstore.persist()
retriever = vectorstore.as_retriever()

C:\Users\ebosjoy\AppData\Local\Temp\ipykernel_24748\1934441728.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
C:\Users\ebosjoy\AppData\Local\Programs\Python\Python310\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# Step 3: Load TinyLlama using Ollama
llm = Ollama(model="tinyllama")

In [ ]:
# Step 4: Create QA Chain
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")

In [ ]:
# Step 5: Ask MarxBot
while True:
    query = input("\nAsk MarxBot: ")
    if query.lower() in ["exit", "quit"]:
        break
    result = qa_chain.run(query)
    print("\n", result)